In [2]:
import math
import os
import numpy as np
import pickle
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets
import matplotlib.pyplot as plt
import torch.nn as nn
import tqdm
import copy
# from util import *

In [3]:
# Set up device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("device:", device)

# Setup seeds
np.random.seed(819)
torch.manual_seed(819)
torch.cuda.manual_seed(819)
torch.backends.cudnn.deterministic = True

# Setup for experiments
dtype = torch.float
alpha = 0.05  # test threshold
batch_size = 100  # size of the batches

N_EPOCHS = 1000
N_TRAIL = 10  # number of trails
N_TEST = 100  # number of test sets
N_TEST_F = 100.0  # number of test sets (float)0
N_PER = 100  # permutation times

# Fetch data
channels = 1  # number of image channels
img_size = 32  # size of each image dimension

device: cuda:0


In [4]:
torch.manual_seed(819)
# Configure data loader
os.makedirs("./data/mnist", exist_ok=True)
dataloader_real_all = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(),
             transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=60000,
    shuffle=True,
)
# dataloader_real_te = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "./data/mnist",
#         train=False,
#         download=True,
#         transform=transforms.Compose(
#             [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#         ),
#     ),
#     batch_size=10000,
#     shuffle=True,
# )

# Obtain MNIST images
for i, (imgs, labels) in enumerate(dataloader_real_all):
    data_real_all = imgs
    label_real_all = labels
# for i, (imgs, labels) in enumerate(dataloader_real_te):
#     data_real_te = imgs
#     label_real_te = labels

# Obtain fake MNIST images
data_fake_all = pickle.load(
    open('./data/Fake_MNIST_data_EP100_N10000.pckl', 'rb'))[0]
# data_fake_all = torch.from_numpy(data_fake_all).float()
data_fake_all = MatConvert(data_fake_all, device, dtype)

NameError: name 'MatConvert' is not defined

In [19]:
data_real_all.shape, data_fake_all.shape

(torch.Size([60000, 1, 32, 32]), torch.Size([10000, 1, 32, 32]))

In [22]:
data_real_all[0], data_fake_all[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9969, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9999, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -0.9939, -0.9950, -0.9976],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9920, -0.9816, -0.9985],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9993, -0.9997, -0.9996]]],
        device='mps:0'))

In [ ]:
torch.cat([data_real_all[0], data_fake_all[0]], dim=0)

tensor([[[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         ...,
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]],

        [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-0.9969, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-0.9999, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         ...,
         [-1.0000, -1.0000, -1.0000,  ..., -0.9939, -0.9950, -0.9976],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9920, -0.9816, -0.9985],
         [-1.0000, -1.0000, -1.0000,  ..., -0.9993, -0.9997, -0.9996]]])

In [1]:
import math
import os
import numpy as np
import pickle
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets
import matplotlib.pyplot as plt
import torch.nn as nn
import tqdm
import copy
from utils import *

# Set up device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("device:", device)

# Setup seeds
np.random.seed(819)
torch.manual_seed(819)
torch.cuda.manual_seed(819)
torch.backends.cudnn.deterministic = True

# Setup for experiments
dtype = torch.float
alpha = 0.05 # test threshold
batch_size = 100 # size of the batches

N_EPOCHS = 2000
N_TRAIL = 10  # number of trails
N_TEST = 100  # number of test sets
N_TEST_F = 100.0  # number of test sets (float)0
N_PER = 100  # permutation times

# Fetch data
channels = 1 # number of image channels
img_size = 32 # size of each image dimension

device: cuda:0


In [2]:
torch.manual_seed(819)
# Configure data loader
os.makedirs("./data/mnist", exist_ok=True)
dataloader_real_all = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=60000,
    shuffle=True,
)

# Obtain MNIST images
for i, (imgs, labels) in enumerate(dataloader_real_all):
    data_real_all = imgs
    label_real_all = labels

# Obtain fake MNIST images
data_fake_all = pickle.load(open('./data/Fake_MNIST_data_EP100_N10000.pckl', 'rb'))[0]
data_fake_all = torch.from_numpy(data_fake_all).float()

In [3]:
# Define the deep network for MMD-D
class Featurizer(nn.Module):
    def __init__(self):
        super(Featurizer, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0)] #0.25
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, 100))

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        feature = self.adv_layer(out)

        return feature

In [4]:
# for n in [100, 200]:
n = 100
summary = []
for kk in tqdm.trange(100):
    n_train = n
    n_test = n

    s1_tr, s1_te, s2_tr, s2_te = sample_mnist_semi(data_real_all, data_fake_all, n_train, n_test, kk=kk)

    S = np.concatenate((s1_tr, s2_tr), axis=0)
    S = MatConvert(S, device, dtype)
    
    torch.manual_seed(kk * 19 + n)
    torch.cuda.manual_seed(kk * 19 + n)
    np.random.seed(seed=1102 * (kk + 10) + n)
    # Initialize deep networks for MMD-D (called featurizer), C2ST-S and C2ST-L (called discriminator)
    featurizer = Featurizer().to(device)
    # Initialize parameters
    epsilonOPT = torch.log(MatConvert(np.random.rand(1) * 10 ** (-10), device, dtype))
    epsilonOPT.requires_grad = True
    sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2*32*32), device, dtype)
    sigmaOPT.requires_grad = True
    sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
    sigma0OPT.requires_grad = True
    
    optimizer_F = torch.optim.Adam(list(featurizer.parameters()) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=0.001)

    dataset = torch.utils.data.TensorDataset(s1_tr, s2_tr)
        
    dataloader_MMD = torch.utils.data.DataLoader(
        dataset, batch_size=100, shuffle=True
    )

    for epoch in range(2000):
        for real_imgs, fake_imgs in dataloader_MMD:
            X = torch.cat([real_imgs, fake_imgs], 0)
            X = X.to(device)
            optimizer_F.zero_grad()
            # Compute output of deep network
            modelu_output = featurizer(X)
            # Compute epsilon, sigma and sigma_0
            ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute Compute J (STAT_u)
            TEMP = MMDu(modelu_output, real_imgs.shape[0], X.view(X.shape[0],-1), sigma, sigma0_u, ep)
            mmd_value_temp = -1 * (TEMP[0])
            mmd_std_temp = torch.sqrt(TEMP[1] + 10 ** (-8))
            STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            # Compute gradient
            STAT_u.backward()
            # Update weights using gradient descent
            optimizer_F.step()

    S_test = np.concatenate((s1_te, s2_te), axis=0)
    S_test = MatConvert(S_test, device, dtype)
    H_MMD = np.zeros(N_TEST)
    for k in range(100):
        H_MMD[k], _, _ = TST_MMD_u(
        featurizer(S_test), n_train, N_PER, S_test.view(S_test.shape[0],-1), sigma, sigma0_u, ep, alpha, k)
    # h, _, _ = TST_MMD_u(
    #     featurizer(S_test), n_train, N_PER, S_test.view(X.shape[0],-1), sigma, sigma0_u, ep, alpha, 3)
    print(H_MMD.sum() / N_TEST_F)
    summary.append(H_MMD.sum() / N_TEST_F)

print(np.mean(summary))

  0%|          | 0/100 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [11]:
# for n in [100, 200]:
n = 100
summary = []
for kk in tqdm.trange(100):
    n_train = n
    n_test = n

    s1_tr, s1_te, s2_tr, s2_te = sample_mnist_semi(
        data_real_all[:4000], data_fake_all[:4000], n_train, n_test, kk=kk)

    S = np.concatenate((s1_tr, s2_tr), axis=0)
    S = MatConvert(S, device, dtype)

    torch.manual_seed(kk * 19 + n)
    torch.cuda.manual_seed(kk * 19 + n)
    np.random.seed(seed=1102 * (kk + 10) + n)
    featurizer = Featurizer().to(device)
    # Initialize parameters
    # epsilonOPT = torch.log(MatConvert(
    #     np.random.rand(1) * 10 ** (-10), device, dtype))
    epsilonOPT = MatConvert(np.random.rand(1) * 10 ** (-8), device, dtype)
    epsilonOPT.requires_grad = True
    sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2*32*32), device, dtype)
    sigmaOPT.requires_grad = True
    sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
    # sigma0OPT = MatConvert(np.ones(1) * np.sqrt(100), device, dtype)
    sigma0OPT.requires_grad = True

    optimizer_F = torch.optim.Adam(list(featurizer.parameters(
    )) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=0.001)

    dataset = torch.utils.data.TensorDataset(s1_tr, s2_tr)

    dataloader_MMD = torch.utils.data.DataLoader(
        dataset, batch_size=100, shuffle=True
    )

    for epoch in range(2000):
        for real_imgs, fake_imgs in dataloader_MMD:

            X = torch.cat([real_imgs, fake_imgs], 0)
            X = X.to(device)
            optimizer_F.zero_grad()

            # Compute output of deep network
            modelu_output = featurizer(X)
            # Compute epsilon, sigma and sigma_0
            # ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
            ep = epsilonOPT
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute Compute J (STAT_u)
            TEMP = MMDu(modelu_output, real_imgs.shape[0], X.view(
                X.shape[0], -1), sigma, sigma0_u, ep)
            mmd_value_temp = -1 * (TEMP[0])
            mmd_std_temp = torch.sqrt(TEMP[1] + 10 ** (-8))
            STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            # Compute gradient
            STAT_u.backward()
            # Update weights using gradient descent
            optimizer_F.step()

    H_MMD = np.zeros(N_TEST)
    S_test = np.concatenate((s1_te, s2_te), axis=0)
    S_test = MatConvert(S_test, device, dtype)
    for k in range(100):
        H_MMD[k], _, _ = TST_MMD_u(
            featurizer(S_test), n_train, N_PER, S_test.view(S_test.shape[0], -1), sigma, sigma0_u, ep, alpha, k)
    print(H_MMD.sum() / N_TEST_F)
    summary.append(H_MMD.sum() / N_TEST_F)
    print(np.mean(summary))

# print(np.mean(summary))

  1%|          | 1/100 [00:16<27:20, 16.57s/it]

0.0
0.0


  2%|▏         | 2/100 [00:43<37:13, 22.79s/it]

0.85
0.425


  3%|▎         | 3/100 [01:18<45:25, 28.10s/it]

...

0.0
0.27602040816326534


 99%|█████████▉| 99/100 [33:36<00:16, 16.17s/it]

0.86
0.2819191919191919


100%|██████████| 100/100 [33:52<00:00, 20.33s/it]

0.94
0.28850000000000003


In [12]:
np.sum(summary)

28.85

In [6]:
# for n in [100, 200]:
n = 200
summary = []
for kk in tqdm.trange(100):
    n_train = n
    n_test = n

    s1_tr, s1_te, s2_tr, s2_te = sample_mnist_semi(
        data_real_all, data_fake_all, n_train, n_test, kk=kk)

    S = np.concatenate((s1_tr, s2_tr), axis=0)
    S = MatConvert(S, device, dtype)

    torch.manual_seed(kk * 19 + n)
    torch.cuda.manual_seed(kk * 19 + n)
    np.random.seed(seed=1102 * (kk + 10) + n)
    featurizer = Featurizer().to(device)
    # Initialize parameters
    epsilonOPT = torch.log(MatConvert(
        np.random.rand(1) * 10 ** (-10), device, dtype))
    epsilonOPT.requires_grad = True
    sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2*32*32), device, dtype)
    sigmaOPT.requires_grad = True
    sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
    sigma0OPT.requires_grad = True

    optimizer_F = torch.optim.Adam(list(featurizer.parameters(
    )) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=0.001)

    dataset = torch.utils.data.TensorDataset(s1_tr, s2_tr)

    dataloader_MMD = torch.utils.data.DataLoader(
        dataset, batch_size=100, shuffle=True
    )

    for epoch in range(2000):
        for real_imgs, fake_imgs in dataloader_MMD:

            X = torch.cat([real_imgs, fake_imgs], 0)
            X = X.to(device)
            optimizer_F.zero_grad()

            # Compute output of deep network
            modelu_output = featurizer(X)
            # Compute epsilon, sigma and sigma_0
            ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute Compute J (STAT_u)
            TEMP = MMDu(modelu_output, real_imgs.shape[0], X.view(
                X.shape[0], -1), sigma, sigma0_u, ep)
            mmd_value_temp = -1 * (TEMP[0])
            mmd_std_temp = torch.sqrt(TEMP[1] + 10 ** (-8))
            STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            # Compute gradient
            STAT_u.backward()
            # Update weights using gradient descent
            optimizer_F.step()

    H_MMD = np.zeros(N_TEST)
    S_test = np.concatenate((s1_te, s2_te), axis=0)
    S_test = MatConvert(S_test, device, dtype)
    for k in range(100):
        H_MMD[k], _, _ = TST_MMD_u(
            featurizer(S_test), n_train, N_PER, S_test.view(S_test.shape[0], -1), sigma, sigma0_u, ep, alpha, k)
    print(H_MMD.sum() / N_TEST_F)
    summary.append(H_MMD.sum() / N_TEST_F)

print(np.mean(summary))

  1%|          | 1/100 [00:32<53:01, 32.13s/it]

0.97


  2%|▏         | 2/100 [01:02<51:14, 31.37s/it]

1.0


  3%|▎         | 3/100 [01:34<50:50, 31.44s/it]

...

1.0


 84%|████████▍ | 84/100 [52:14<09:13, 34.59s/it]

1.0


 84%|████████▍ | 84/100 [52:16<09:57, 37.34s/it]


KeyboardInterrupt: 